# yield 协程

In [ ]:
import time

def A():
    while 1:
        print('----A-----')
        time.sleep(0.5)
        yield 1

def B():
    while 1:
        print('----B-----')
        time.sleep(0.5)
        next(a)

a = A()

B()

In [6]:
# 斐波那契数列

def fib(max):
    n, a, b = 0, 0, 1
    while n < max:
        yield b   # 这种一边循环一边计算的机制，节省了存储空间，提高了运行效率。
        a, b = b, a + b
        n = n + 1
        
list(fib(10))

[1, 1, 2, 3, 5, 8, 13, 21, 34, 55]

# greenlet

In [1]:
from greenlet import greenlet
import time

def A():
    while 1:
        print('-------A-------')
        time.sleep(0.5)
        g2.switch()

def B():
    while 1:
        print('-------B-------')
        time.sleep(0.5)
        g1.switch()

g1 = greenlet(A)  #创建协程g1
g2 = greenlet(B)

In [2]:
g1.switch()  #跳转至协程g1

-------A-------
-------B-------
-------A-------
-------B-------
-------A-------
-------B-------
-------A-------
-------B-------
-------A-------
-------B-------
-------A-------
-------B-------


KeyboardInterrupt: 

# 3. gevent

greenlet可以实现协程，不过每一次都要人为的去指向下一个该执行的协程，显得太过麻烦。python还有一个比greenlet更强大的并且能够自动切换任务的模块gevent
gevent每次遇到io操作，需要耗时等待时，会自动跳到下一个协程继续执行。

In [3]:
import gevent

def A():
    while 1:
        print('-------A-------')
        gevent.sleep(1) #用来模拟一个耗时操作，注意不是time模块中的sleep

def B():
    while 1:
        print('-------B-------')
        gevent.sleep(0.5)  #每当碰到耗时操作，会自动跳转至其他协程

g1 = gevent.spawn(A) # 创建一个协程
g2 = gevent.spawn(B)
g1.join()  #等待协程执行结束
g2.join()

-------A-------
-------B-------
-------B-------
-------A-------
-------B-------
-------B-------
-------A-------
-------B-------
-------B-------
-------A-------
-------B-------
-------B-------
-------A-------
-------B-------
-------B-------


KeyboardInterrupt
2019-04-28T15:16:49Z


KeyboardInterrupt: 

# 4.协程gevent完成回显服务器：

In [4]:
import gevent
from gevent import monkey, socket

monkey.patch_all()  #有IO才做时需要这一句

s = socket.socket(2, 1)  #用的都是gevent模块中的socket,但用法一样
s.setsockopt(1, 2, 1)
s.bind(('', 8889))
s.listen(1024)


def func_accept():
    while 1:
        cs, userinfo = s.accept()
        print('来了一个客户' + str(userinfo))
        g = gevent.spawn(func_recv, cs)  #每当有用户连接,增加一条协程


def func_recv(cs):
    while 1:
        recv_data = cs.recv(1024)
        print(recv_data)  #程谁堵塞了,便会跳转至其他协程
        if len(recv_data) > 0:
            cs.send(recv_data)
        else:
            cs.close()
            break


g1 = gevent.spawn(func_accept)
g1.join()

KeyboardInterrupt
2019-07-10T07:53:28Z


KeyboardInterrupt: 

# asyncio

In [ ]:
import asyncio

async def compute(x, y):
    print("Compute %s + %s ..." % (x, y))
    await asyncio.sleep(1.0)
    return x + y

async def print_sum(x, y):
    result = await compute(x, y)
    print("%s + %s = %s" % (x, y, result))

loop = asyncio.get_event_loop()
loop.run_until_complete(print_sum(1, 2))
loop.close()
